In [ ]:
!pip install langchain langchain_community langchain-google-genai python-dotenv langchain_experimental langchain_chroma langchainhub pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 27.6 MB/s eta 

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Update the path to your PDF file
loader = PyPDFLoader("/content/customer_support.pdf")
data = loader.load()  # entire PDF is loaded as a single Document

# Verify the data
print(data)


[Document(metadata={'source': '/content/customer_support.pdf', 'page': 0}, page_content=' \nCustomer Support Ticket Examples \nTicket 1: Password Reset Request \n• Customer Query: "I forgot my password and can\'t log in to my account. Can you help me \nreset it?" \n• Response: "Dear [Customer Name], we have received your request to reset your password. \nPlease click on the following link to reset your password: [Reset Link]. If you encounter any \nissues, feel free to contact us. Thank you!" \nTicket 2: Billing Issue \n• Customer Query: "I was charged twice for my subscription this month. Can you please look \ninto this?" \n• Response: "Dear [Customer Name], we apologize for the inconvenience caused by the \ndouble charge. Our billing team is currently investigating the issue. You will receive a refund \nfor the extra charge within 3-5 business days. Thank you for your patience. " \nTicket 3: Product Defect \n• Customer Query: "The product I received is defective. It doesn\'t turn on.

In [ ]:
len(data)

2

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  6


In [ ]:
import os
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Load environment variables from a .env file
load_dotenv()
os.environ["GOOGLE_API_KEY"] = api_key

# Initialize the embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
print(vector[:5])


[0.05168594419956207, -0.030764883384108543, -0.03062233328819275, -0.02802734263241291, 0.01813093200325966]


In [ ]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

retrieved_docs = retriever.invoke("experiencing frequent crashes when using your software?")

In [ ]:
len(retrieved_docs)

5

In [ ]:
print(retrieved_docs[0].page_content)

• Customer Query: "I think it would be great if your app had a dark mode feature. Is this 
something you are considering?" 
• Response: "Dear [Customer Name], thank you for your suggestion! We appreciate your 
feedback and are always looking to improve our app. We have forwarded your request to our 
development team for consideration. Stay tuned for updates!" 
Ticket 7: Technical Issue 
• Customer Query: "I'm experiencing frequent crashes when using your software. Can you 
help me resolve this?" 
• Response: "Dear [Customer Name], we apologize for the inconvenience caused by the 
crashes. Please provide us with the following details: your operating system, software 
version, and any error messages you receive. Our technical support team will assist you in 
resolving the issue. " 
Ticket 8: Subscription Cancellation 
• Customer Query: "I would like to cancel my subscription. How can I do that?" 
• Response: "Dear [Customer Name], to cancel your subscription, please log in to your


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "my name is Nithesh.i have received an damaged product what should i do now ?"})
print("RAG Output:", response["answer"])


RAG Output: Dear Nithesh, we are sorry to hear about the damaged product. Please provide us with your order number and a brief description of the damage. We will arrange for a replacement or a refund as per your preference.

